In [1]:
%matplotlib inline
import numpy as np
from numpy.linalg import norm
from math import sqrt
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

In [3]:
def projection_ell(point, a, b):
    k = sqrt(1 / ((point[0]/a)**2+(point[1]/b)**2))
    prj = np.linspace(k, k, num=DIM) * point
    #print(point)
    #print(prj)
    return prj

In [45]:
### SETTINGS
MAX_COUNT_ITER = 500
EPS = 10**(-5)
DIM = 2
h = np.array([0.001, 0.001], dtype = float)

###CONSTRAINTS for ellipsoid x^2/a^2 + y^2/b^2 + z^2/c^2 <= 1
A = 10.0   #actually it's 1/a^2
B = 2.0   #actually it's 1/b^2

### TARGET FUNCTIONS
def f_test(x):
    return 2.0 * x[0] + 3.0 * x[1]

def f_myvar(x):
    return 2.0 * x[0] + 3.0 * x[1]

f = f_test
x0 = np.array([1, 1], dtype = float)

In [44]:
def df1(f,x,h):
    return (f([x[0] + h, x[1]]) - f([x[0] - h, x[1]])) / (2.0 * h)
def df2(f,x,h):
    return (f([x[0], x[1] + h]) - f([x[0], x[1] - h])) / (2.0 * h)

def grad(f,x,h):
    return np.array([df1(f,x,h[0]), df2(f,x,h[1])], dtype = float)

def alpha_split(f,x,step,b=1,l=0.5,q=0.1):
    alpha = b
    #while (f(x+alpha*step) >= f(x)):
    while f(x+alpha*step) > f(x) + q * alpha * grad(f,x,h).dot(step):
        alpha *= l
    #print(alpha)
    return alpha

### STOP CONDITIONS
def stop1(x1,x2,k):
    d = norm(x2-x1)
    #plt.xlabel('iteration')
    #plt.ylabel('|| x_new - x_old || ')
    #plt.scatter(k, d)
    return d<=EPS

def stop2(f,x1,x2,k):
    d = abs(f(x2)-f(x1))
    #plt.xlabel('iteration')
    #plt.ylabel('| f(x_new) - f(x_old) | ')    
    #plt.scatter(k, d)
    return d<=EPS

###PROJECTIONS
def projection_ellipse(point): 
    if point[0]**2/A + point[1]**2/B <= 1:
        return point
    dist = lambda x: norm(x - point)
    ellipse = ( {'type': 'ineq', 'fun': lambda x: 1 - A * x[0]**2 - B * x[1]**2}) 
    return minimize(dist, (0.0, 0.0), method='SLSQP', constraints = ellipse).x

def projection_ball(point, a = np.zeros((DIM)), r=1):
    #prj = a + r * (point - a)*1.0/norm(point - a)
    nrm = norm(point - a)
    prj = np.array([a[i] + r * (point[i]-a[i])/nrm for i in range(DIM)])
    #print(point)
    #print(prj)
    return prj

def my_projection_ellipse(point):
    if point[0]**2/A + point[1]**2/B <= 1:
        return point
    temp = projection_ball(point) 
    #temp = projection_ball([sqrt(A) * t[0], sqrt(B) * t[1]]) 
    prj = np.array([sqrt(1/A) * temp[0], sqrt(1/B) * temp[1]])
    #print(temp)        #print(x_new)     
    #print(norm(temp))  #print(A * x_new[0]**2 + B * x_new[1]**2)
    return prj

# МЕТОД ПРОЕКЦІЇ ГРАДІЄНТА


In [46]:
def projection_gradient_method(f,x0,h):
    fout = open('output.txt', 'w')
    fout.write('The initial point is ({}, {})\n\n'.format(*x0))
    x_new = x0
    k = 0
    while (k<300):
        k += 1 
        x_old = x_new    
        step = - grad(f,x_old,h)
        alpha = alpha_split(f,x_old,step)
        t = x_old + alpha * step
        fout.write('{iter:>3}. alpha = {al:<5.3f},   x_{iter:<3} = ({:>7.4f}, {:>7.4f}),   '.format(iter=k, *t, al=alpha))
        x_new = projection_ellipse(t)
        fout.write('prx_{iter:<3} = ({:>7.4f}, {:>7.4f}),   f(x_{iter:<}) = {f:>7.4f}\n'.format(iter=k, *x_new, f=f(x_new)))
        if (stop1(x_old,x_new,k) and stop2(f,x_old,x_new,k)):
            break
    print('Approximate solution found in {} iterations'.format(k))
    print('> Approximate   x*  = ({:>8.5f}, {:>8.5f})'.format(*x_new))
    print('> Approximate f(x*) = {:>8.5f}'.format(f(x_new)))
    fout.write('\nThe approximate solution of the problem is ({:>8.5f}, {:>8.5f})\n'.format(*x_new))
    fout.write('The value of function in this point is {:>8.5f}\n'.format(f(x_new)))
    fout.close()
    return x_new

minim = projection_gradient_method(f,x0,h)
#plt.show()
print()

Approximate solution found in 4 iterations
> Approximate   x*  = (-0.09035, -0.67763)
> Approximate f(x*) = -2.21359



# МЕТОД ПРОЕКЦІЇ ГРАДІЄНТА З ФОРМУЛОЮ ДЛЯ ОБЧИСЛЕННЯ ПРОЕКЦІЇ


In [41]:
def projection_gradient_method_f(f,x0,h):
    fout = open('output.txt', 'w')
    fout.write('The initial point is ({}, {})\n\n'.format(*x0))
    x_new = x0
    k = 0
    while (k<MAX_COUNT_ITER):
        k += 1 
        x_old = x_new    
        step = - grad(f,x_old,h)
        alpha = alpha_split(f,x_old,step)
        t = x_old + alpha * step
        fout.write('{iter:>3}. alpha = {al:<5.3f},   x_{iter:<3} = ({:>7.4f}, {:>7.4f}),   '.format(iter=k, *t, al=alpha))
        x_new = my_projection_ellipse(t)
        fout.write('prx_{iter:<3} = ({:>7.4f}, {:>7.4f}),   f(x_{iter:<}) = {f:>7.4f}\n'.format(iter=k, *x_new, f=f(x_new)))
        if (stop1(x_old,x_new,k) and stop2(f,x_old,x_new,k)):
            break
    print('Approximate solution found in {} iterations'.format(k))
    print('> Approximate   x*  = ({:>8.5f}, {:>8.5f})'.format(*x_new))
    print('> Approximate f(x*) = {:>8.5f}'.format(f(x_new)))
    fout.write('\nThe approximate solution of the problem is ({:>8.5f}, {:>8.5f})\n'.format(*x_new))
    fout.write('The value of function in this point is {:>8.5f}\n'.format(f(x_new)))
    fout.close()
    return x_new

minim = projection_gradient_method_f(f,x0,h)
#plt.show()
print()

Approximate solution found in 6 iterations
> Approximate   x*  = (-0.16263, -0.60643)
> Approximate f(x*) = -2.14455

